In [14]:
##complex data와 똑같이 40ms로 parsing했음 데이터 4배라고 생각하고 뻥튀기시켜봄 정확도는 10ms와 비슷하게 나옴

import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
from sklearn import svm
from datetime import datetime
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score 
import glob


#filename queue
filename_queue=glob.glob('./rawdata/data[1-4].csv')
#Create return path
#return_path='./parsed/data.csv'
#create_file=open(return_path,'w')

In [8]:
#Parsing data

training_set=[]
test_set=[]
training_label=[]
test_label=[]
X=[]
y=[]
index=[] # 2,8,11
label_count=np.zeros(16)
traininglabel_count=np.zeros(16)
testlabel_count=np.zeros(16)

#return_file=open(return_path,'a')
#writer=csv.writer(return_file)
#writer.writerow()
n_data=[]

for filename in filename_queue:
    file=open(filename, newline='')
    
    reader=csv.reader(file)
    header=next(reader)
    
    for row in reader:
        row[4:]=[float(i) for i in row[4:]]
        date=datetime.strptime(row[0],'%Y/%m/%d')
        msec=row[1:3]
        labl=int(row[3])
        if labl not in index:
            label_count[labl]+=1
            for x in range(4):
                X_acc=list(row[4*i+4+x] for i in range(150))
                Y_acc=list(row[4*i+604+x] for i in range(150))
                Z_acc=list(row[4*i+1204+x] for i in range(150))
                X_gyro=list(row[4*i+1804+x] for i in range(150))
                Y_gyro=list(row[4*i+2404+x] for i in range(150))
                Z_gyro=list(row[4*i+3004+x] for i in range(150))

                window=np.array([[X_acc[0:25],X_acc[25:50],X_acc[50:75],X_acc[75:100],X_acc[100:125],X_acc[125:150]],
                    [Y_acc[0:25],Y_acc[25:50],Y_acc[50:75],Y_acc[75:100],Y_acc[100:125],Y_acc[125:150]],
                    [Z_acc[0:25],Z_acc[25:50],Z_acc[50:75],Z_acc[75:100],Z_acc[100:125],Z_acc[125:150]],
                    [X_gyro[0:25],X_gyro[25:50],X_gyro[50:75],X_gyro[75:100],X_gyro[100:125],X_gyro[125:150]],
                    [Y_gyro[0:25],Y_gyro[25:50],Y_gyro[50:75],Y_gyro[75:100],Y_gyro[100:125],Y_gyro[125:150]],
                    [Z_gyro[0:25],Z_gyro[25:50],Z_gyro[50:75],Z_gyro[75:100],Z_gyro[100:125],Z_gyro[125:150]]])

                #5 features  what else?2
                window_mean=window.mean(axis=-1)
                window_stddev=window.std(axis=-1)
                window_median=np.median(window,axis=-1)
                window_percent25=np.percentile(window,25,axis=-1)
                window_percent75=np.percentile(window,75,axis=-1)

                window_feature=np.array([[window_mean],[window_stddev],[window_median],[window_percent25],[window_percent75]])
                window_feature=window_feature.reshape(180)

                #for cross validation
                X.append(window_feature)
                y.append(labl)
                #수동으로 X, y 나누기
                if (label_count[labl]%5!=1):
                    training_set.append(window_feature)
                    training_label.append(labl)
                else:
                    test_set.append(window_feature)
                    test_label.append(labl)


from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix

clf = KNeighborsClassifier(n_neighbors=5)
scores=cross_val_score(clf,X,y,cv=10,scoring='accuracy')
print(scores.mean())


#test_pred3=clf.predict(test_set)
#print(accuracy_score(test_pred3,test_label))
#print(confusion_matrix(test_pred3, test_label))
#onehot_pred3=test_pred2.reshape(-1,1)
#onehot_pred3=ohc.fit_transform(onehot_pred3).toarray()
#print(onehot_pred3)

k_range=range(1,31)
k_scores=[]
for k in k_range :
    knn = KNeighborsClassifier(n_neighbors=k)
    scores=cross_val_score(knn,X,y,cv=10,scoring='accuracy')
    k_scores.append(scores.mean())
print(k_scores)

plt.plot(k_range,k_scores)
plt.xlabel('value of k for knn')
plt.ylabel('accuracy')

from sklearn.model_selection import GridSearchCV

k_range=range(1,31)
param_grid=dict(n_neighbors=k_range)
grid=GridSearchCV(knn,param_grid,cv=10,scoring='accuracy')

grid.fit(X,y)

(grid.cv_results_)

scores=cross_val_score(KNeighborsClassifier(1),X,y,cv=10,scoring='accuracy')


In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

clf=RandomForestClassifier()
param_grid={
 'n_estimators':[100,150,200],
   # 'max_features':['auto','sqrt','log2'],
    'max_depth':[10,12,None]
    #'criterion':['gini','entropy']
}

grid=GridSearchCV(estimator=clf,param_grid=param_grid,cv=10,scoring='accuracy')

In [10]:
grid.fit(X,y)

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             ii

In [11]:
grid.cv_results_

{'mean_fit_time': array([0.48374104, 0.75854883, 0.98243542, 0.49089046, 0.74347527,
        1.017524  , 0.5376415 , 0.79332376, 1.03282115]),
 'std_fit_time': array([0.0057305 , 0.04151035, 0.02928669, 0.00629604, 0.00683577,
        0.02412534, 0.02091229, 0.03993779, 0.04373222]),
 'mean_score_time': array([0.00617254, 0.00922012, 0.01197698, 0.0062017 , 0.00912912,
        0.01209314, 0.00660017, 0.01061411, 0.01251502]),
 'std_score_time': array([1.45965269e-04, 5.20664594e-04, 1.69921323e-04, 6.91769607e-05,
        9.23868795e-05, 2.63493286e-04, 9.51517942e-04, 2.22400021e-03,
        1.50434157e-03]),
 'param_max_depth': masked_array(data=[10, 10, 10, 12, 12, 12, None, None, None],
              mask=[False, False, False, False, False, False, False, False,
                    False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[100, 150, 200, 100, 150, 200, 100, 150, 200],
              mask=[False, False, False, False, False, F

In [12]:
grid.best_params_

{'max_depth': 12, 'n_estimators': 150}

In [15]:
clf1=RandomForestClassifier(80)
scores=cross_val_score(clf1,X,y,cv=10,scoring='accuracy')
print(scores.mean())

0.8653724237122906


#SVM
#prepare the data
#prepare the model
clf = sklearn.svm.SVC(gamma='scale',tol=0.1)
scores=cross_val_score(clf,X,y,cv=10,scoring='accuracy')
print(scores.mean())


param_grid={
 #'kernel':['linear','poly','rbf','sigmoid'], rbf to be best acc
    'C':[0.001,0.01,0.1,1,10],
    'gamma' : [0.001, 0.01, 0.1, 1]

}

grid=GridSearchCV(estimator=clf,param_grid=param_grid,cv=10,scoring='accuracy')

grid.fit(X,y)

grid.best_params_

grid.cv_results_

clf1=sklearn.svm.SVC(gamma='scale',tol=0.1)
scores=cross_val_score(clf1,X,y,cv=10,scoring='accuracy')
print(scores.mean())

In [16]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
bagging = BaggingClassifier(KNeighborsClassifier(),max_samples=0.5, max_features=0.5)

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier

ETC=ExtraTreesClassifier(300)
DTC=DecisionTreeClassifier()








In [17]:
param_grid={
 'n_estimators':[100,200,300],
   # 'max_features':['auto','sqrt','log2'],
    'max_depth':[10,14,None]
    #'criterion':['gini','entropy']
}

grid=GridSearchCV(estimator=ETC,param_grid=param_grid,cv=10,scoring='accuracy')

In [18]:
grid.fit(X,y)


/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=ExtraTreesClassifier(bootstrap=False, class_weight=None,
                                            criterion='gini', max_depth=None,
                                            max_features='auto',
                                            max_leaf_nodes=None,
                                            min_impurity_decrease=0.0,
                                            min_impurity_split=None,
                                            min_samples_leaf=1,
                                            min_samples_split=2,
                                            min_weight_fraction_leaf=0.0,
                                            n_estimators=300, n_jobs=None,
                                            oob_score=False, random_state=None,
                                            verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': [10, 14, No

In [19]:
grid.cv_results_

{'mean_fit_time': array([0.1304548 , 0.2451071 , 0.37172558, 0.12869022, 0.2500546 ,
        0.37867482, 0.12856832, 0.25821536, 0.38903143]),
 'std_fit_time': array([0.01510675, 0.00636484, 0.01972231, 0.01065756, 0.00131767,
        0.00325476, 0.00113647, 0.00296044, 0.00371133]),
 'mean_score_time': array([0.00663681, 0.01247132, 0.02039464, 0.00644343, 0.0122649 ,
        0.01810007, 0.00640903, 0.01250131, 0.01832995]),
 'std_score_time': array([5.55292674e-04, 5.59358668e-04, 2.78713105e-03, 6.45199661e-05,
        1.14290568e-04, 1.58925139e-04, 5.37528104e-05, 3.08114890e-04,
        2.29103289e-04]),
 'param_max_depth': masked_array(data=[10, 10, 10, 14, 14, 14, None, None, None],
              mask=[False, False, False, False, False, False, False, False,
                    False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[100, 200, 300, 100, 200, 300, 100, 200, 300],
              mask=[False, False, False, False, False, F

In [21]:
grid.best_params_

{'max_depth': 14, 'n_estimators': 200}

In [20]:
test_pred_proba2=ETC.predict_proba(test_set)
array=np.array(test_pred_proba2)
newpred3=np.zeros(len(test_pred4))
for x in range(len(test_pred_proba2)):
    if(np.max(test_pred_proba2[x])>0.2):
        newpred3[x]=test_pred4[x]
    else:
        newpred3[x]=-1
        i=0
        #print(np.shape(np.argmax(test_pred_proba[x])))
print(np.shape(newpred3))
print(newpred3)
print(accuracy_score(newpred3,test_label))

NotFittedError: This ExtraTreesClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.